In [1]:
import numpy as np
from pint.core import roundfloat as rf

In [2]:
def split(a):
    tmp = a * (2**27 + 1)
    x = tmp - (tmp - a)
    y = a - x
    return x, y

In [3]:
def twoproduct(a, b):
        x = np.zeros(a.shape)
        y = np.zeros(a.shape)
        aB2 = abs(a) > 2. ** 996
        a[aB2] *= 2. ** (-28)
        b[aB2] *= 2. ** 28
        bB2 = abs(b) > 2. ** 996
        a[bB2] *= 2. ** 28
        b[bB2] *= 2. ** (-28)
        aHm, aLm = split(a)
        bHm, bLm = split(b)
        x = a * b
        xB2 = abs(x) > 2. ** 1023
        aH = aHm[xB2]
        aL = aLm[xB2]
        bH = bHm[xB2]
        bL = bLm[xB2]
        y[xB2] = aL * bL - ((((x[xB2] * 0.5) - (aH * 0.5) * bH) * 2. - aL * bH) - aH * bL)
        xB2r = ~xB2
        aH = aHm[xB2r]
        aL = aLm[xB2r]
        bH = bHm[xB2r]
        bL = bLm[xB2r]
        y[xB2r] = aL * bL - (((x[xB2r] - aH * bH) - aL * bH) - aH * bL)
        return x, y

In [4]:
pntwoproduct = rf.twoproduct
def twoproduct_old(a, b):
    x = np.zeros(a.shape)
    y = np.zeros(a.shape)
    lenx = len(x)
    for i in range(0, lenx):
        for j in range(0, lenx):
            x[i][j], y[i][j] = pntwoproduct(a[i][j], b[i][j])
    return x, y

In [5]:
size = 1000
a = np.random.rand(size, size)
b = np.random.rand(size, size)

In [6]:
x, y = twoproduct(a, b)
old_x, old_y = twoproduct_old(a, b)

In [7]:
# もし、違う項があれば検出される
print(x[x != old_x])
print(y[y != old_y])

[]
[]


In [8]:
%timeit twoproduct(a, b)

10 loops, best of 3: 76.8 ms per loop


In [9]:
%timeit twoproduct_old(a, b)

1 loop, best of 3: 4.22 s per loop
